# Exportar os json com os dados mais importantes/relevantes
- business: business_id; name; city; latitude; longitude; stars; categories
- review: review_id; user_id; business_id; stars; date; # texto não será utilizado provavelmente, se for so mudar
- user: nada
- checkin: nada
- tip: user_id; business_id; date # não será utilizado pois não tem stars

In [2]:
import json
import pandas as pd
import numpy as np
import pyarrow

In [3]:
fbusiness=open("business.json")
all_data = list()
for i, line in enumerate(fbusiness):  
    # json to dict
    business_json = json.loads(line)
    # add to the data collection
    all_data.append([business_json['business_id'],
                     business_json['name'],
                     business_json['city'],
                     business_json['latitude'],
                     business_json['longitude'],
                     business_json['stars'],
                     business_json['categories']])
# create the DataFrame
df = pd.DataFrame(all_data, columns=['business_id', 'name', 'city','latitude', 'longitude', 'stars', 'categories'])

df.head()

,business_id,name,city,latitude,longitude,stars,categories
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,Phoenix,33.522143,-112.018481,3.0,"Golf, Active Life"
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,Mississauga,43.605499,-79.652289,2.5,"Specialty Food, Restaurants, Dim Sum, Imported..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,Charlotte,35.092564,-80.859132,4.0,"Sushi Bars, Restaurants, Japanese"
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,Goodyear,33.455613,-112.395596,5.0,"Insurance, Financial Services"
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,Charlotte,35.190012,-80.887223,4.0,"Plumbing, Shopping, Local Services, Home Servi..."


In [4]:
df.to_feather('business.feather')

In [5]:
freview=open("review.json")
all_data = list()
for i, line in enumerate(freview):  
    # json to dict
    business_json = json.loads(line)
    # add to the data collection
    all_data.append([business_json['review_id'],
                     business_json['user_id'],
                     business_json['business_id'],
                     business_json['stars'],
                     business_json['date']])
# create the DataFrame
df = pd.DataFrame(all_data, columns=['review_id', 'user_id', 'business_id','stars', 'date'])

df.head()

,review_id,user_id,business_id,stars,date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,2013-05-07 04:34:36
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,2017-01-14 21:30:33
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,2016-11-09 20:09:03
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,2018-01-09 20:56:38
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,2018-01-30 23:07:38


In [6]:
df.to_feather('review.feather')